Cargando dataset de entrenamiento

Se carga el data set de viejos trabajos y papers online.

In [1]:
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os

In [2]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    fp = open(path, 'rb')
    iterPDF = iter(PDFPage.get_pages(fp, set(), maxpages=0, password="",caching=True, check_extractable=True))
    next(iterPDF)

    for page in iterPDF:
        interpreter.process_page(page)
        text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
directorio_data_sets = r'D:\TP NLP\DataSet'
files = os.listdir(directorio_data_sets)
documentos = []
for f in files:
    if os.path.isfile(os.path.join(directorio_data_sets, f)) and f.endswith('.pdf'):
        documentos.append(convert_pdf_to_txt(os.path.join(directorio_data_sets, f)).replace('\n',''))
corpus = '.'.join(documentos)

print("Documentos leidos: " + str(len(documentos)))

Documentos leidos: 110


Preprocesado del corpus

In [4]:
from nltk.corpus import stopwords
from nltk import re
from nltk import word_tokenize
import unicodedata

def preprocess_corpus(corpus):
    corpus = corpus.lower() 
    
    #eliminar signos de puntuacion
    words = [word for word in word_tokenize(corpus) if re.search("\w", word)]
    
    #eliminar stop words
    corpus_sin_stop_words = ' '.join(filter(lambda x: x not in stopwords.words('spanish'), words))
    
    #eliminar numeros
    clean_corpus = ''.join([i for i in corpus_sin_stop_words if not i.isdigit()])
    
    #eliminar caracteres unicode
    clean_corpus = unicodedata.normalize("NFKD", clean_corpus).encode("ascii","ignore").decode("ascii")
    clean_corpus = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', clean_corpus)
    
    #reemplazar espacios consecutivos por uno solo
    clean_corpus = re.compile(r"\s+").sub(" ", clean_corpus).strip()
        
    return clean_corpus

In [5]:
clean_corpus = preprocess_corpus(corpus) #limpia el corpus inicial

In [6]:
clean_corpus

'cuestionario .- que elementos hacen resurgir fuerza idea economia experiencia .- defina caracterice experiencia diferencias producto servicio .- explique grafique dimensiones campos experiencia .- describa impresiones distintas dimensiones .- ejemplos distintos reales si conoce invente experiencias estimulacion .- personalizacion masiva explique progresion valor .- cuales ventajas empresa personalizacion masiva .- describa tipos personalizacion masiva .- que aporta ciberespacio tema sacrificio cliente respuestas .- tres elementos poder tecnologia actua facilitador creciente intensidad competencia hace diferenciar experiencias prosperidad decir hoy dia busca menos rutina sorpresas .- hablamos producto referimos ejemplo taza cafe etc... si referimos servicio hablamos conjunto actividades decir customizado cliente hablamos experiencia referimos pagamos disfrutar serie eventos acondicionados atraparnos gozar manera distinta solo comprar producto adquirir servicio hecho usan productos serv